In [ ]:
from autoPyTorch.api.tabular_classification import TabularClassificationTask
from autoPyTorch.api.tabular_regression import TabularRegressionTask
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer, fetch_california_housing
import numpy as np

# Пример классификации
# -------------------
# Загрузка данных
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Создание задачи классификации
api = TabularClassificationTask()

# Настройка и запуск оптимизации
api.search(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    optimize_metric='accuracy',
    total_walltime_limit=3600,  # 1 час
    func_eval_time_limit_secs=300,  # 5 минут на одну модель
    memory_limit=8192,  # 8 ГБ
    n_jobs=-1,  # Использовать все ядра
    ensemble_size=5,  # Размер ансамбля
    include_preprocessors=['feature_type', 'scaling', 'imputation'],
    resampling_strategy='cv',
    resampling_strategy_args={'folds': 5}
)

# Получение результатов
y_pred = api.predict(X_test)
print(f"Test accuracy: {np.mean(y_pred == y_test)}")

# Получение вероятностей
y_proba = api.predict_proba(X_test)

# Получение лучшей модели
best_model = api.get_best_model()

# Пример регрессии
# --------------
# Загрузка данных
X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Создание задачи регрессии
api_reg = TabularRegressionTask()

# Настройка и запуск оптимизации
api_reg.search(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    optimize_metric='r2',
    total_walltime_limit=3600,
    func_eval_time_limit_secs=300,
    memory_limit=8192,
    n_jobs=-1
)

# Получение результатов
y_pred = api_reg.predict(X_test)
from sklearn.metrics import r2_score, mean_squared_error
print(f"Test R2: {r2_score(y_test, y_pred)}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")

In [ ]:
from autoPyTorch.api.tabular_classification import TabularClassificationTask
from sklearn.model_selection import train_test_split
import numpy as np

# Создание задачи с настройкамими ансамбля
api = TabularClassificationTask(
    ensemble_size=10,  # Размер ансамбля
    ensemble_nbest=5,  # Использовать 5 лучших моделей
    max_models_on_disc=20,  # Хранить до 20 моделей
    seed=42,
    include_components={
        'feature_preprocessor': ['pca', 'quantile_transformer', 'polynomial'],
        'classifier': ['random_forest', 'extra_trees', 'adaboost', 'mlp', 'lgb']
    }
)

# Запуск поиска с настройками для разнообразия моделей
api.search(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    optimize_metric='balanced_accuracy',
    total_walltime_limit=7200,  # 2 часа
    func_eval_time_limit_secs=300,
    memory_limit=8192,
    ensemble_size=10,
    ensemble_strategy='bagging',  # Использование бэггинга
    resampling_strategy='cv',
    resampling_strategy_args={'folds': 5}
)

# Получение результатов ансамбля
y_pred = api.predict(X_test)
y_proba = api.predict_proba(X_test)

# Получение отдельных моделей из ансамбля
ensemble_models = api.get_ensemble_models()

# Использование отдельной модели из ансамбля
single_model = ensemble_models[0]
single_prediction = single_model.predict(X_test)

In [ ]:
from autoPyTorch.api.nlp_task import NLPTask
from sklearn.model_selection import train_test_split
import pandas as pd

# Предположим, у нас есть текстовые данные
texts = ["This is a positive text", "This is a negative example", ...]
labels = [1, 0, ...]

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2)

# Создание NLP задачи
api = NLPTask(
    language='english',
    task_type='classification',
    transformer_models=['bert-base-uncased', 'roberta-base'],
    max_seq_length=128,
    include_preprocessing_pipelines=['standard']
)

# Запуск поиска
api.search(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    optimize_metric='accuracy',
    total_walltime_limit=7200,
    func_eval_time_limit_secs=600,
    memory_limit=12288,  # 12 ГБ
    n_jobs=1  # Обычно для трансформеров используется 1 процесс
)

# Получение результатов
y_pred = api.predict(X_test)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet,  # Для регрессии
    LogisticRegression, SGDClassifier  # Для классификации
)

# Генерация данных для регрессии
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=1000, n_features=20, noise=0.1, random_state=42)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 1. Линейная регрессия
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
print(f"Linear Regression R²: {r2_score(y_test, y_pred_lr)}")
print(f"Linear Regression RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_lr))}")

# Визуализация коэффициентов
plt.figure(figsize=(10, 6))
plt.bar(range(X.shape[1]), lr.coef_)
plt.xlabel('Feature Index')
plt.ylabel('Coefficient Value')
plt.title('Linear Regression Coefficients')
plt.tight_layout()
plt.show()

# 2. Гребневая регрессия (Ridge)
ridge = Ridge(alpha=1.0)  # alpha - параметр регуляризации
ridge.fit(X_train_scaled, y_train)
y_pred_ridge = ridge.predict(X_test_scaled)
print(f"Ridge Regression R²: {r2_score(y_test, y_pred_ridge)}")

# 3. Лассо регрессия 
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_scaled, y_train)
y_pred_lasso = lasso.predict(X_test_scaled)
print(f"Lasso Regression R²: {r2_score(y_test, y_pred_lasso)}")

# 4. Elastic Net
elastic = ElasticNet(alpha=0.1, l1_ratio=0.5)  # l1_ratio - баланс между L1 и L2
elastic.fit(X_train_scaled, y_train)
y_pred_elastic = elastic.predict(X_test_scaled)
print(f"ElasticNet R²: {r2_score(y_test, y_pred_elastic)}")

# Оптимизация гиперпараметров для Ridge с помощью GridSearchCV
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(
    Ridge(), 
    param_grid, 
    cv=5, 
    scoring='neg_mean_squared_error',
    verbose=1
)
grid_search.fit(X_train_scaled, y_train)
print(f"Best Ridge alpha: {grid_search.best_params_}")
print(f"Best Ridge MSE: {-grid_search.best_score_}")

# Для классификации
# ----------------
# Генерация данных для классификации
from sklearn.datasets import make_classification
X_cls, y_cls = make_classification(
    n_samples=1000, n_features=20, n_informative=10, 
    n_redundant=5, random_state=42
)

# Разделение данных
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=42
)

# Нормализация
X_train_cls_scaled = scaler.fit_transform(X_train_cls)
X_test_cls_scaled = scaler.transform(X_test_cls)

# 1. Логистическая регрессия
logreg = LogisticRegression(
    C=1.0,                  # Обратное к регуляризации
    penalty='l2',           # Тип регуляризации (L2)
    solver='liblinear',     # Алгоритм оптимизации
    max_iter=1000,          # Максимальное число итераций
    random_state=42
)
logreg.fit(X_train_cls_scaled, y_train_cls)
y_pred_logreg = logreg.predict(X_test_cls_scaled)
print(f"Logistic Regression Accuracy: {accuracy_score(y_test_cls, y_pred_logreg)}")
print(classification_report(y_test_cls, y_pred_logreg))

# Вероятности для ROC-AUC
y_proba_logreg = logreg.predict_proba(X_test_cls_scaled)[:, 1]

# 2. SGD Classifier
sgd_cls = SGDClassifier(
    loss='log_loss',    # Функция потери (логистическая)
    penalty='elasticnet',  # Регуляризация
    alpha=0.0001,       # Сила регуляризации
    l1_ratio=0.15,      # Соотношение L1/L2
    max_iter=1000,
    random_state=42
)
sgd_cls.fit(X_train_cls_scaled, y_train_cls)
y_pred_sgd = sgd_cls.predict(X_test_cls_scaled)
print(f"SGD Classifier Accuracy: {accuracy_score(y_test_cls, y_pred_sgd)}")

# Оптимизация гиперпараметров для логистической регрессии
param_grid_logreg = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}
grid_search_logreg = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=1000), 
    param_grid_logreg, 
    cv=5, 
    scoring='accuracy',
    verbose=1
)
grid_search_logreg.fit(X_train_cls_scaled, y_train_cls)
print(f"Best Logistic Regression params: {grid_search_logreg.best_params_}")
print(f"Best Logistic Regression accuracy: {grid_search_logreg.best_score_}")